In [ ]:
!pip install ultralytics
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.6/793.6 kB 6.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [5]:
import cv2
import json
import albumentations as A
import os

# Define the augmentation pipeline with only flipping and rotating
test_augmentations = A.Compose([
    A.HorizontalFlip(p=0.5),  # Apply random horizontal flip
    A.Rotate(limit=45, p=0.5),  # Apply random rotation up to 45 degrees
], bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']))

# Directories
test_dir = 'path_to_test_images'
annotations_file = 'path_to_annotations.json'  # COCO-style annotations
output_img_dir = 'path_to_augmented_images'
output_anno_file = 'path_to_augmented_annotations.json'

# Make sure the output directories exist
os.makedirs(output_img_dir, exist_ok=True)

# Load annotations
with open(annotations_file) as f:
    annotations = json.load(f)

# Initialize a list to hold updated annotations
augmented_annotations = []

# Loop through each image and its corresponding annotation
for img_info in annotations['images']:
    img_path = os.path.join(test_dir, img_info['file_name'])
    image = cv2.imread(img_path)
    
    # Get bounding boxes for the current image
    img_id = img_info['id']
    bboxes = []
    category_ids = []
    
    for anno in annotations['annotations']:
        if anno['image_id'] == img_id:
            bboxes.append(anno['bbox'])
            category_ids.append(anno['category_id'])

    # Apply augmentation (flipping and rotating)
    augmented = test_augmentations(image=image, bboxes=bboxes, category_ids=category_ids)
    
    # Save augmented image
    output_img_path = os.path.join(output_img_dir, img_info['file_name'])
    cv2.imwrite(output_img_path, augmented['image'])
    
    # Create new annotation entry for the augmented image
    for bbox, category_id in zip(augmented['bboxes'], augmented['category_ids']):
        new_annotation = {
            "image_id": img_id,  # You may need to update or generate new IDs
            "category_id": category_id,
            "bbox": bbox,
            "area": bbox[2] * bbox[3],  # width * height
            "iscrowd": 0,
            "id": len(augmented_annotations) + 1  # unique ID for each annotation
        }
        augmented_annotations.append(new_annotation)

# Save updated annotations
new_annotations = annotations.copy()
new_annotations['annotations'] = augmented_annotations

with open(output_anno_file, 'w') as f:
    json.dump(new_annotations, f)

print(f"Augmented images saved in {output_img_dir}")
print(f"Updated annotations saved as {output_anno_file}")



Saved augmented image: C:\Users\gbo10\OneDrive\measurement_paper_images\960540-DETR\test\images\aug\20230920_115022_jpg.rf.1584201d92d308bc9dd9da7709d9c93c.jpg
Saved augmented image: C:\Users\gbo10\OneDrive\measurement_paper_images\960540-DETR\test\images\aug\20230920_115121_jpg.rf.2c2f7c8ca8ecdae3a396ef7ce0eac0d8.jpg
Saved augmented image: C:\Users\gbo10\OneDrive\measurement_paper_images\960540-DETR\test\images\aug\20230920_115133_jpg.rf.20ae2c63bf177f046f1b84f260a73dd5.jpg
Saved augmented image: C:\Users\gbo10\OneDrive\measurement_paper_images\960540-DETR\test\images\aug\20230920_115139_jpg.rf.5a0b4e0acc14bbe027b734f83bef8298.jpg
Saved augmented image: C:\Users\gbo10\OneDrive\measurement_paper_images\960540-DETR\test\images\aug\20230920_115202_jpg.rf.631c84cb155a4194d494d857278059b7.jpg
Saved augmented image: C:\Users\gbo10\OneDrive\measurement_paper_images\960540-DETR\test\images\aug\20230920_115213_jpg.rf.27a736e2cdff2b242ada8b3df99bca05.jpg
Saved augmented image: C:\Users\gbo10\On

In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('prawns/measurement paper/yolov8-only_31.12_-segment_segment.pt:v31', type='model')
artifact_dir = artifact.download()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Downloading large artifact yolov8-only_31.12_-segment_segment.pt:v31, 263.64MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:18.0


In [ ]:
from YoloToWandbKeypoint import WandbCallback

from pathlib import Path
import wandb
import re
# Load a COCO-pretrained RT-DETR-l model
import wandb


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import importlib
import yolov8l16

importlib.reload(yolov8l16)

<module 'yolov8l16' from '/content/yolov8l16.py'>

In [ ]:
from ultralytics import RTDETR

# Load a COCO-pretrained RT-DETR-l model
model = RTDETR("rtdetr-l.pt")

100%|██████████| 137M/137M [00:00<00:00, 322MB/s]


In [ ]:

wandb_logger = WandbCallback(
        model,project='measurement paper',run_name='rtdetr-drone-600-25',tags=['detection','drone','rtdetr-l']
    )

for event, callback_fn in wandb_logger.callbacks.items():
    model.add_callback(event, callback_fn)

## train notice yaml addresses ##

In [ ]:
model.train(data="/content/drive/MyDrive/colab experiments/to colab only 31-12 segment with 76/data.yaml",copy_paste=0.15,mixup=0.15, epochs=300, imgsz=640, seed=42,batch=16,patience=50)




In [6]:
from ultralytics import RTDETR  


model = RTDETR(r"C:\Users\gbo10\OneDrive\measurement_paper_images\weights- 28.08\best\RT_DETR 8 .pt",)

metrics = model.val(save_json=True, data=r"c:\Users\gbo10\OneDrive\measurement_paper_images\nnnnnnnnnn\dataset.yaml",seed=42, plots=True,split='test')

Ultralytics YOLOv8.2.73  Python-3.9.7 torch-2.1.2+cpu CPU (Intel Core(TM) i7-10750H 2.60GHz)
rt-detr-l summary: 502 layers, 31,985,795 parameters, 0 gradients, 103.4 GFLOPs


val: Scanning C:\Users\gbo10\OneDrive\measurement_paper_images\nnnnnnnnnn\aug-test\labels... 79 images, 1 backgrounds, 0 corrupt: 100%|██████████| 79/79 [00:00<00:00, 863.37it/s]

val: New cache created: C:\Users\gbo10\OneDrive\measurement_paper_images\nnnnnnnnnn\aug-test\labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [01:10<00:00, 14.13s/it]


                   all         79        561      0.841      0.776      0.812      0.287
Speed: 9.3ms preprocess, 858.7ms inference, 0.0ms loss, 1.2ms postprocess per image
Saving c:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\detect\val16\predictions.json...
Results saved to c:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\detect\val16


In [4]:
from ultralytics import YOLO

model=RTDETR(r"C:\Users\gbo10\OneDrive\measurement_paper_images\weights- 28.08\best\RT_DETR 8 .pt")

metrics = model.val(save_json=True, data=r"C:\Users\gbo10\OneDrive\measurement_paper_images\current from roboflow -28.8-first sense\circle pond.v4i.yolov8\dataset.yaml",seed=42, plots=True,split='test')

Ultralytics YOLOv8.2.73  Python-3.9.7 torch-2.1.2+cpu CPU (Intel Core(TM) i7-10750H 2.60GHz)
rt-detr-l summary: 502 layers, 31,985,795 parameters, 0 gradients, 103.4 GFLOPs


val: Scanning C:\Users\gbo10\OneDrive\measurement_paper_images\current from roboflow -28.8-first sense\circle pond.v4i.yolov8\test\labels.cache... 79 images, 1 backgrounds, 0 corrupt: 100%|██████████| 79/79 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [01:29<00:00, 17.84s/it]


                   all         79        456      0.801      0.811      0.817      0.285
Speed: 6.1ms preprocess, 1071.0ms inference, 0.0ms loss, 0.7ms postprocess per image
Saving c:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\detect\val15\predictions.json...
Results saved to c:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\detect\val15
